In [1]:
import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.pricing as pricing
from oandapyV20.contrib.requests import MarketOrderRequest
import oandapyV20.endpoints.orders as orders
import pandas as pd
import configparser
import json
import numpy as np
import time

# get accountID and token from configuration file
config = configparser.ConfigParser()
config.read('oanda.cfg')
accountID = config['oanda']['account_id']
access_token = config['oanda']['access_token']

api = oandapyV20.API(access_token=access_token)
units = 105
position = 0

params ={
          "instruments": "EUR_USD"
        }

r = pricing.PricingStream(accountID=accountID, params=params)
rv = api.request(r)
dfr= pd.DataFrame()

maxrecs = 0
for ticks in rv:
    #df = pd.DataFrame(ticks)
    #df1 = df1.append(df,ignore_index=True)
    #print ('123')
    #print (json.dumps(ticks),";end")
    if ticks['type'] != 'PRICE':
        continue
    
    df = pd.DataFrame({'Time':ticks['time'],'closeoutAsk': ticks['closeoutAsk']},index = [maxrecs])
    dfr = dfr.append(df,ignore_index=True,sort=False)
    #print (json.dumps(ticks),";end")
    maxrecs += 1
    
    dfr["closeoutAsk"] = dfr.closeoutAsk.astype(float)
    dfr['returns'] = np.log(dfr['closeoutAsk'] / dfr['closeoutAsk'].shift(1))  # 41
    # derives the positioning according to the momentum strategy
    if maxrecs < 3:
        continue
        

    dfr['position'] = np.sign(dfr['returns'].rolling(2).mean())  # 42
    
    if dfr['position'].iloc[-1] == 1:  # 43
        # go long
        if position == 0:  # 44
            r = orders.OrderCreate(accountID,
                                   data=MarketOrderRequest(instrument=params["instruments"],units=units).data)
            rv = api.request(r)
            
            print('go long, order placed at ' + rv["orderFillTransaction"]["price"])
        elif position == -1:  # 46

            r = orders.OrderCreate(accountID,
                                   data=MarketOrderRequest(instrument=params["instruments"],units=2*units).data)
            rv = api.request(r)
            print('go long, double order placed at ' + rv["orderFillTransaction"]["price"])
        position = 1  # 48
    elif dfr['position'].iloc[-1] == -1:  # 49
        # go short
        if position == 0:  # 50
            r = orders.OrderCreate(accountID,
                                   data=MarketOrderRequest(instrument=params["instruments"],units=-units).data)
            rv = api.request(r)                
            print('go short, order placed at ' + rv["orderFillTransaction"]["price"])
        elif position == 1: # 52
            r = orders.OrderCreate(accountID,
                                   data=MarketOrderRequest(instrument=params["instruments"],units=(-2)*units).data)
            rv = api.request(r)
            print('go short, double order placed at ' + rv["orderFillTransaction"]["price"])
        position = -1  # 54
    if maxrecs == 250:  # 55
        # close out the position
        if position == 1:  # 56
            # sell
            r = orders.OrderCreate(accountID,
                                   data=MarketOrderRequest(instrument=params["instruments"],units=-units).data)
            rv = api.request(r)
            print ("sell and close the position")
        elif position == -1:  # 58
            # buy
            r = orders.OrderCreate(accountID,
                                   data=MarketOrderRequest(instrument=params["instruments"],units=units).data)
            rv = api.request(r)
            print ("buy and close the position")
        break
    
    
    #if maxrecs == 10:
    #    r.terminate("maxrecs records received")
        

ModuleNotFoundError: No module named 'oandapyV20'

In [2]:
dfr

,Time,closeoutAsk,position,returns
0,2020-04-28T08:40:29.207747539Z,1.08548,NaN,NaN
1,2020-04-28T08:40:32.009720040Z,1.08546,NaN,-0.000018
2,2020-04-28T08:40:34.599439738Z,1.08545,-1.0,-0.000009
3,2020-04-28T08:40:34.660742397Z,1.08545,-1.0,0.000000
4,2020-04-28T08:40:35.212330807Z,1.08545,0.0,0.000000
...,...,...,...,...
245,2020-04-28T08:44:23.914592819Z,1.08526,1.0,0.000046
246,2020-04-28T08:44:24.124809444Z,1.08526,1.0,0.000000
247,2020-04-28T08:44:28.172522085Z,1.08532,1.0,0.000055
248,2020-04-28T08:44:28.399000002Z,1.08538,1.0,0.000055
